In [1]:
import pandas as pd
import pickle
import weighted_sentiment_functions_final as wsff
import stock_price_calculation_trials as spct
from datetime import datetime
from tqdm import tqdm

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# To do: organise the print statements, write code to save output lists into pickle files.

In [3]:
all_data = pd.read_csv('final_all_mdna_data.csv', sep='.@', header=None, names=['DELETE', 'CIK', 'TICKER', 'DATE', 'TEXT'], dtype={'CIK': object}, engine='python', parse_dates=[3], infer_datetime_format=True, encoding='utf-8')
all_data

,DELETE,CIK,TICKER,DATE,TEXT
0,NaN,0000002488,AMD,2019-02-08,Our 2018 financial results demonstrate the suc...
1,NaN,0000002488,AMD,2018-02-27,2017 was an important year for AMD as we launc...
2,NaN,0000002488,AMD,2017-02-21,As we continued to focus on our strategy to im...
3,NaN,0000002488,AMD,2016-02-18,We faced a challenging business environment in...
4,NaN,0000002488,AMD,2015-02-19,"Net revenue for 2014 was $5.5 billion, an incr..."
...,...,...,...,...,...
234,NaN,0000766704,WELL,2009-03-02,"Health Care REIT, Inc., an S&P 500 company, is..."
235,NaN,0000766704,WELL,2008-02-28,"Health Care REIT, Inc. is an equity real esta..."
236,NaN,0000766704,WELL,2007-03-01,"Health Care REIT, Inc. is a self-administered,..."
237,NaN,0000766704,WELL,2006-03-10,"Health Care REIT, Inc. is a self-administered,..."


In [4]:
new_all_data = all_data.drop(['DELETE'], axis=1)
new_all_data

,CIK,TICKER,DATE,TEXT
0,0000002488,AMD,2019-02-08,Our 2018 financial results demonstrate the suc...
1,0000002488,AMD,2018-02-27,2017 was an important year for AMD as we launc...
2,0000002488,AMD,2017-02-21,As we continued to focus on our strategy to im...
3,0000002488,AMD,2016-02-18,We faced a challenging business environment in...
4,0000002488,AMD,2015-02-19,"Net revenue for 2014 was $5.5 billion, an incr..."
...,...,...,...,...
234,0000766704,WELL,2009-03-02,"Health Care REIT, Inc., an S&P 500 company, is..."
235,0000766704,WELL,2008-02-28,"Health Care REIT, Inc. is an equity real esta..."
236,0000766704,WELL,2007-03-01,"Health Care REIT, Inc. is a self-administered,..."
237,0000766704,WELL,2006-03-10,"Health Care REIT, Inc. is a self-administered,..."


In [5]:
# Check dataframe for any missing values:
# new_all_data.isnull().values.any()
# new_all_data.isnull().sum().sum()

In [6]:
# load the trained classifier model from disk
filename = 'finalised_classifier_model.sav'
model = pickle.load(open(filename, 'rb'))

To-do: Might need to modify this, instead round each value maybe?
In regular sent analysis you get 0, 0.5, 1. With averaging justifications you get between 0 and 1, i.e. scaled/weighted.
Below is more like weighted mode aggregated. Whereas above 'weighted' function is like weighted mean aggregated.
But mode doesn't make much sense because it gets rid of the weight, so it is more like checking if 5 mined justifications can
capture the full sentiment of the entire document.
Read what you wrote in your thesis and then clarify all these sentiment score terms below!!!!!!

In [7]:
weighted_sentiments = []
for i, row in tqdm(new_all_data.iterrows(), desc='Getting sentiment scores', unit='Filings'):
    weighted_sentiment = wsff.get_weighted_sentiment(row['TEXT'], model, clustering_model='Agglomerative')
    weighted_sentiments.append(weighted_sentiment)
print(weighted_sentiments)
new_all_data['weighted_sentiments'] = weighted_sentiments

unweighted_sentiments = []
for i, row in tqdm(new_all_data.iterrows(), desc='Getting sentiment scores', unit='Filings'):
    unweighted_sentiment = wsff.get_unweighted_sentiment(row['TEXT'], model, clustering_model='Agglomerative')
    unweighted_sentiments.append(unweighted_sentiment)
print(unweighted_sentiments)
new_all_data['unweighted_sentiments'] = unweighted_sentiments

Getting sentiment scores: 239Filings [09:29,  2.38s/Filings]


[0.80000000000000004, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.80000000000000004, 0.90000000000000002, 0.80000000000000004, 1.0, 1.0, 0.90000000000000002, 0.90000000000000002, 0.80000000000000004, 0.90000000000000002, 1.0, 1.0, 1.0, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 1.0, 1.0, 0.80000000000000004, 0.80000000000000004, 0.90000000000000002, 0.80000000000000004, 0.80000000000000004, 0.80000000000000004, 0.80000000000000004, 0.90000000000000002, 1.0, 0.80000000000000004, 0.69999999999999996, 0.80000000000000004, 0.80000000000000004, 0.80000000000000004, 1.0, 1.0, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.90000000000000002, 0.80000000000000004, 0.80

Getting sentiment scores: 239Filings [09:13,  2.32s/Filings]


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [8]:
full_sentiments_mean = []

for i, row in tqdm(new_all_data.iterrows(), desc='Getting sentiment scores', unit='Filings'):
    full_sentiment_mean = wsff.get_full_sentiment_mean(row['TEXT'], model)
    full_sentiments_mean.append(full_sentiment_mean)
print(full_sentiments_mean)
new_all_data['full_mean'] = full_sentiments_mean

full_sentiments_mean_unrounded = []

for i, row in tqdm(new_all_data.iterrows(), desc='Getting sentiment scores', unit='Filings'):
    full_sentiment_mean_unrounded = wsff.get_full_sentiment_mean_unrounded(row['TEXT'], model)
    full_sentiments_mean_unrounded.append(full_sentiment_mean_unrounded)
print(full_sentiments_mean_unrounded)
new_all_data['full_mean_unrounded'] = full_sentiments_mean_unrounded

full_sentiments_mode = []

for i, row in tqdm(new_all_data.iterrows(), desc='Getting sentiment scores', unit='Filings'):
    full_sentiment_mode = wsff.get_full_sentiment_mode(row['TEXT'], model)
    full_sentiments_mode.append(full_sentiment_mode)
print(full_sentiments_mode)
new_all_data['full_mode'] = full_sentiments_mode

Getting sentiment scores: 239Filings [00:04, 51.05Filings/s]


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

Getting sentiment scores: 239Filings [00:04, 50.69Filings/s]


[0.9285714285714286, 0.92592592592592593, 0.91176470588235292, 0.86904761904761907, 0.93181818181818177, 1.0, 0.92307692307692313, 0.91304347826086951, 0.95454545454545459, 0.95945945945945943, 0.96666666666666667, 0.90740740740740744, 0.9285714285714286, 0.967741935483871, 0.9375, 1.0, 0.97297297297297303, 0.97297297297297303, 0.97222222222222221, 0.96052631578947367, 0.97368421052631582, 0.95945945945945943, 0.97297297297297303, 0.97297297297297303, 0.97297297297297303, 0.97368421052631582, 0.97499999999999998, 0.97560975609756095, 0.98780487804878048, 0.96052631578947367, 0.95833333333333337, 0.984375, 0.96153846153846156, 0.9375, 0.94594594594594594, 0.91025641025641024, 0.90476190476190477, 0.95348837209302328, 0.91891891891891897, 0.9375, 0.90000000000000002, 0.90000000000000002, 0.8666666666666667, 0.97727272727272729, 0.95454545454545459, 0.95454545454545459, 0.95833333333333337, 0.94444444444444442, 0.9464285714285714, 0.97826086956521741, 0.95833333333333337, 0.95652173913043

Getting sentiment scores: 239Filings [00:04, 50.40Filings/s]


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

Note the time difference in weighted vs full sentiment scores:
weighted:
Getting sentiment scores: 239Filings [09:29,  2.38s/Filings]
Getting sentiment scores: 239Filings [09:13,  2.32s/Filings]

Full:
Getting sentiment scores: 239Filings [00:04, 51.05Filings/s]
Getting sentiment scores: 239Filings [00:04, 50.69Filings/s]
Getting sentiment scores: 239Filings [00:04, 50.40Filings/s]

Likely caused by the embeddings extraction time.

In [9]:
new_all_data.head()

,CIK,TICKER,DATE,TEXT,weighted_sentiments,unweighted_sentiments,full_mean,full_mean_unrounded,full_mode
0,0000002488,AMD,2019-02-08,Our 2018 financial results demonstrate the suc...,0.8,1.0,1.0,0.928571,1.0
1,0000002488,AMD,2018-02-27,2017 was an important year for AMD as we launc...,0.9,1.0,1.0,0.925926,1.0
2,0000002488,AMD,2017-02-21,As we continued to focus on our strategy to im...,0.9,1.0,1.0,0.911765,1.0
3,0000002488,AMD,2016-02-18,We faced a challenging business environment in...,0.9,1.0,1.0,0.869048,1.0
4,0000002488,AMD,2015-02-19,"Net revenue for 2014 was $5.5 billion, an incr...",0.9,1.0,1.0,0.931818,1.0


Trial outputs for all the below stock calculations:
1:
[23.049999237060547, 331.04998779296875, 26.155519485473633]
[Timestamp('2019-02-08 00:00:00'), Timestamp('2019-01-31 00:00:00'), Timestamp('2010-11-18 00:00:00')]

2:
[52.259998321533203, 517.46002197265625, 16.585308074951172]
[Timestamp('2020-02-10 00:00:00'), Timestamp('2020-01-31 00:00:00'), Timestamp('2011-11-18 00:00:00')]
[27.090000152587891, 371.19000244140625, 27.930322647094727]
[Timestamp('2019-05-08 00:00:00'), Timestamp('2019-04-30 00:00:00'), Timestamp('2011-02-18 00:00:00')]

3:
[126.72451215316249, 56.308727096605168, -36.589643787566928]
[17.527119519516933, 12.125061509907129, 6.7855779450558877]

In [10]:
def get_current_stock_price_date(ticker, date):
    given_date = date
    while True:
        try:
            stock_price = spct.get_stock_adj_close(ticker, given_date)
            break
        except KeyError:
            given_date = spct.add_days(given_date,1)
    return given_date, stock_price

stock_prices = []
given_dates = []


    
for i, row in tqdm(new_all_data.iterrows(), desc='Getting stock prices', unit='Filings'):
    given_date, stock_price = get_current_stock_price_date(row['TICKER'], row['DATE'])
    stock_prices.append(stock_price)
    given_dates.append(given_date)
    
print(stock_prices)
print(given_dates)

Getting stock prices: 239Filings [04:21,  1.09s/Filings]


[23.049999237060547, 12.529999732971191, 14.0, 1.9099999666213989, 3.059999942779541, 3.7000000476837158, 2.5999999046325684, 7.4600000381469727, 9.1800003051757813, 7.929999828338623, 2.119999885559082, 7.130000114440918, 14.789999961853027, 40.619998931884766, 17.959999084472656, 14.529999732971191, 1626.22998046875, 1429.949951171875, 827.46002197265625, 587.0, 354.52999877929688, 358.69000244140625, 272.760009765625, 179.46000671386719, 171.13999938964844, 125.41000366210938, 58.819999694824219, 75.19000244140625, 40.330001831054688, 39.189998626708984, 34.75, 42.939998626708984, 331.04998779296875, 387.5, 324.45999145507813, 161.08000183105469, 177.77000427246094, 125.08000183105469, 85.019996643066406, 63.569999694824219, 28.89457893371582, 24.875455856323242, 24.219139099121094, 15.710981369018555, 16.892948150634766, 18.17902946472168, 15.811397552490234, 13.947021484375, 10.736077308654785, 8.7257328033447266, 6.4457430839538574, 8.8771677017211914, 9.9546632766723633, 57.1461

[Timestamp('2019-02-08 00:00:00'), Timestamp('2018-02-27 00:00:00'), Timestamp('2017-02-21 00:00:00'), Timestamp('2016-02-18 00:00:00'), Timestamp('2015-02-19 00:00:00'), Timestamp('2014-02-18 00:00:00'), Timestamp('2013-02-21 00:00:00'), Timestamp('2012-02-24 00:00:00'), Timestamp('2011-02-18 00:00:00'), Timestamp('2010-02-19 00:00:00'), Timestamp('2009-02-24 00:00:00'), Timestamp('2008-02-26 00:00:00'), Timestamp('2007-03-01 00:00:00'), Timestamp('2006-02-27 00:00:00'), Timestamp('2005-03-01 00:00:00'), Timestamp('2004-03-09 00:00:00'), Timestamp('2019-02-01 00:00:00'), Timestamp('2018-02-02 00:00:00'), Timestamp('2017-02-10 00:00:00'), Timestamp('2016-01-29 00:00:00'), Timestamp('2015-01-30 00:00:00'), Timestamp('2014-01-31 00:00:00'), Timestamp('2013-01-30 00:00:00'), Timestamp('2012-02-01 00:00:00'), Timestamp('2011-01-28 00:00:00'), Timestamp('2010-01-29 00:00:00'), Timestamp('2009-01-30 00:00:00'), Timestamp('2008-02-11 00:00:00'), Timestamp('2007-02-16 00:00:00'), Timestamp('20

In [30]:
# Check to make sure it isn't SBUX out of date 2019, index 209 should now return SBUX 2018-11-16
print(new_all_data.iloc[209])
print(given_dates[209])
date_one_yearsbux = spct.add_years(given_dates[209], 1)
print(date_one_yearsbux) # Should be 2019-11-16

CIK                                                             0000829224
TICKER                                                                SBUX
DATE                                                   2018-11-16 00:00:00
TEXT                     Starbucks results for fiscal 2018 reflect the ...
weighted_sentiments                                                    0.9
unweighted_sentiments                                                    1
full_mean                                                                1
full_mean_unrounded                                               0.945122
full_mode                                                                1
Name: 210, dtype: object
2018-11-16 00:00:00
2019-11-16 00:00:00


In [33]:
# To-do: create 6 month window as well.
# Getting an error at index 209: issue was SBUX 2019 is in Nov, so next year is Nov 2020, hasn't happened yet.
stock_prices_next_year = []
given_dates_next_year = []
stock_prices_three_months = []
given_dates_three_months = []

for i, row in tqdm(new_all_data.iterrows(), desc='Getting stock prices', unit='Filings'):
    date_one_year = spct.add_years(given_dates[i], 1)
    given_date, stock_price = get_current_stock_price_date(row['TICKER'], date_one_year)
    stock_prices_next_year.append(stock_price)
    given_dates_next_year.append(given_date)
    
for i, row in tqdm(new_all_data.iterrows(), desc='Getting stock prices', unit='Filings'):
    date_three_months = spct.add_months(given_dates[i], 3)
    given_date, stock_price = get_current_stock_price_date(row['TICKER'], date_three_months)
    stock_prices_three_months.append(stock_price)
    given_dates_three_months.append(given_date)

print(stock_prices_next_year)
print(given_dates_next_year)
print(stock_prices_three_months)
print(given_dates_three_months)


Getting stock prices: 0Filings [00:00, ?Filings/s]
Getting stock prices: 1Filings [00:02,  2.09s/Filings]
Exception in thread Thread-15:
Traceback (most recent call last):
  File "C:\Users\Kevin\Anaconda3\envs\thesis\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\Kevin\Anaconda3\envs\thesis\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\Kevin\Anaconda3\envs\thesis\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

Getting stock prices: 237Filings [05:36,  1.42s/Filings]


IndexError: list index out of range

In [ ]:
percent_changes_one_year = []
percent_changes_three_months = []

for old, new in zip(stock_prices, stock_prices_next_year):
    change = spct.percent_change(old,new)
    percent_changes_one_year.append(change)
    
for old, new in zip(stock_prices, stock_prices_three_months):
    change = spct.percent_change(old,new)
    percent_changes_three_months.append(change)

new_all_data['% change 1 year'] = percent_changes_one_year
new_all_data['% change 3 months'] = percent_changes_three_months

print(percent_changes_one_year)
print(percent_changes_three_months)

In [ ]:
new_all_data.head()

In [ ]:
new_all_data.to_pickle("./full_data_df_1.pkl")

Choosing spearmean versus pearson correlation:
'I noted in a comment to the post that usually spearmanr() is used for ranks and it does not seem like you are correlating ranked data. I'm making that assessment based on non-integral values of the 2 sequences-usually ranks are non-negative integral values.

Given the second issue (as it seems) I'd recommend using pearsonr(). Provided your version of scipy is 1.3.X or greater, the error message is informative and tells you exactly this issue:'

Note: Set p-value to 0.05 or less to be statistically significant. 
'The null hypothesis is that the two variables are uncorrelated. The p-value is a number between zero and one that represents the probability that your data would have arisen if the null hypothesis were true.'

In [ ]:
# To-do: format printed lines with % stuff to make it more organised, google how to print with even spacing between
from scipy import special, stats

def logit_correlate(sentiments, percent_changes):
    normalised_sentiments = special.logit(sentiments)
    r, p_value = stats.pearsonr(normalised_sentiments, percent_changes)
    return r, p_value

for sentiment in ['weighted_sentiments', 'unweighted_sentiments', 'full_mean', 'full_mean_unrounded', 'full_mode']:
    r_1yr, p_value_1yr = logit_correlate(new_all_data[sentiment], new_all_data['% change 1 year'])
#     print(sentiment, '1 year', 'r=', r_1yr, 'p=', p_value_1yr)
    print(f'{sentiment:10} 1 year r={r_1yr:10} p={p_value_1yr:10}')
print('\n')
for sentiment in ['weighted_sentiments', 'unweighted_sentiments', 'full_mean', 'full_mean_unrounded', 'full_mode']:
    r_3month, p_value_3month = logit_correlate(new_all_data[sentiment], new_all_data['% change 3 months'])
#     print(sentiment, '3 months', 'r=', r_3month, 'p=', p_value_3month)
    print(f'{sentiment:10} 3 months r={r_3month:10} p={p_value_3month:10}')
    

In [ ]:
# To-do: maybe leave x axis unlabelled as it means nothing.
import numpy as np
import matplotlib.pyplot as plt

t = new_all_data.index
data1 = new_all_data['weighted_sentiments']
data2 = new_all_data['% change 1 year']

fig, ax1 = plt.subplots()

color = 'tab:purple'
ax1.set_xlabel('index')
ax1.set_ylabel('sentiment scores', color=color)
ax1.plot(t, data1, color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_ylim(0, 1)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('% changes in stock price', color=color)  # we already handled the x-label with ax1
ax2.plot(t, data2, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()